In [1]:
import bs4
from typing import List
import requests
import os
import tqdm

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
def get_image_urls(url:str)->List[str]:
    """
    Scrapes all the image urls belongs to the given episode
    and returns them as a list.

    """
    resp = requests.get(url)
    soup = bs4.BeautifulSoup(resp.content,'html.parser')
    
    content = soup.find('div',{'id':'_imageList'})
    images = content.find_all('img')
    result = []
    for i in images:
        result.append(i.attrs['data-url'])
    return result

In [3]:
def download_images(image_list,path)->None:
    """
    Downloads all the images to the given path.
    
    Parameters:
    -----------
    image_list : List of image url
    path: Path to the folder, to download images.
    """
    for index,image in enumerate(image_list):
        
        resp = requests.get(image,headers={'referer':'https://www.webtoons.com'})
        
        with open(os.path.join(path,f'{str(index)}',),'wb') as img:
            img.write(resp.content)

In [4]:
def get_daily_urls(day='MONDAY'):
    """
    Scrapes the mangas URL from the main page from 
    https://www.webtoons.com/en/dailySchedule
    
    Parameters:
    -----------
    day: A day desired to be scraped.
    """
    result = []
    resp = requests.get('https://www.webtoons.com/en/dailySchedule')
    soup = bs4.BeautifulSoup(resp.content,'html.parser')
    r = soup.find('div',{'class',f'_list_{day}'})
    items = r.find_all('li')
    for i in items:
        title = i.find('p',{'class','subj'}).text
        episode_url = i.a['href']
        result.append(episode_url)
    return result

In [5]:
def download_episode_return_next(url:str,parent_folder)-> None:
    """
    Download a manga episode from a given URL and returns the
    next episode URL if it exist.
    
    Parameters:
    -----------
    url: Manga URL to download
    parent_folder: Path for the download folder.
    """
    print(f'Downloading.. {url}')
    
    resp = requests.get(url)
    soup = bs4.BeautifulSoup(resp.content,'html.parser')
    episode_name = soup.find('h1',{'class':'subj_episode'}).text.strip()
    
    path = os.path.join(parent_folder,episode_name)
    
    images = get_image_urls(url)
    os.mkdir(path)
    
    download_images(images,path)
    
    next_episode = soup.find('a',{'class':'pg_next'})['href']
    
    return next_episode
    
    
    

In [6]:
def download_all_episodes(url:str)->None:
    """
    Starts downloading the all episode of manga
    starting from the given Manga URL.
    
    Parameters:
    -----------
    url: Manga episode URL
    """
    resp = requests.get(url)
    soup = bs4.BeautifulSoup(resp.content,'html.parser')
    manga_title = soup.find('div',{'class':'subj_info'}).a.attrs['title']
    
    os.mkdir(manga_title)
    next_episode = download_episode_return_next(url,manga_title)
    while next_episode:
        next_episode = download_episode_return_next(next_episode,manga_title)
        
    

In [7]:
def get_first_episode_url(url:str)->str:
    """
    From the manga description page finds the 
    first episode URL and returns it.
    
    Parameters:
    -----------
    url: Manga description page URL
    
    """
    resp = requests.get(url)
    soup = bs4.BeautifulSoup(resp.content,'html.parser')
    result = soup.find('a',{'id':'_btnEpisode'})['href']
    return result

In [8]:
monday = get_daily_urls('MONDAY')
tuesday = get_daily_urls('TUESDAY')
wednesday = get_daily_urls('WEDNESDAY')
thursday = get_daily_urls('THURSDAY')
friday = get_daily_urls('FRIDAY')
saturday = get_daily_urls('SATURDAY')
sunday = get_daily_urls('SUNDAY')

In [11]:
manga_url = get_first_episode_url(monday[1])
manga_url

'https://www.webtoons.com/en/fantasy/morgana-and-oz/episode-1/viewer?title_no=2964&episode_no=1'

In [ ]:
download_all_episodes(manga_url)

Downloading.. https://www.webtoons.com/en/fantasy/morgana-and-oz/episode-1/viewer?title_no=2964&episode_no=1
Downloading.. https://www.webtoons.com/en/fantasy/morgana-and-oz/episode-2/viewer?title_no=2964&episode_no=2
Downloading.. https://www.webtoons.com/en/fantasy/morgana-and-oz/episode-3/viewer?title_no=2964&episode_no=3
